In [1]:
!pip install mrjob

In [2]:
!pip install pyarrow

In [23]:
%%file task2.py

from mrjob.job import MRJob
from mrjob.step import MRStep

class MRLinesCount(MRJob):
    def steps(self):
        return [
            MRStep(mapper=self.mapper, combiner=self.combiner, reducer=self.reducer_cnt),
            MRStep(reducer=self.reducer_max)
        ]

    def mapper(self, _, line):
        result = line.split('\"')
        yield (result[3], result[-2])

    def combiner(self, character, lines):
        yield (character, max(lines, key=len))

    def reducer_cnt(self, character, lines):
        yield None, (character, max(lines, key=len))

    def reducer_max(self, _, lines):
        for line in sorted(lines, key=lambda x: len(x[1]), reverse=True):
            yield line
        
if __name__ == "__main__":
    MRLinesCount.run()

Overwriting task2.py


In [8]:
file_names = ['SW_EpisodeIV.txt', 'SW_EpisodeV.txt', 'SW_EpisodeVI.txt']

In [24]:
for file_name in file_names:
    output_file = 'Result_'+ file_name
    !python3 task2.py $file_name > $output_file

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/task2.root.20231206.051747.529599
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/task2.root.20231206.051747.529599/output
Streaming final output from /tmp/task2.root.20231206.051747.529599/output...
Removing temp directory /tmp/task2.root.20231206.051747.529599...
No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/task2.root.20231206.051747.828516
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/task2.root.20231206.051747.828516/output
Streaming final output from /tmp/task2.root.20231206.051747.828516/output...
Removing temp directory /tmp/task2.root.20231206.051747.828516...
No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/task2.root.20231206.051748.117743
Running step 1 of 2...
Running 

In [25]:
!python3 task2.py SW_EpisodeIV.txt SW_EpisodeV.txt SW_EpisodeVI.txt > Result_EpisodeAll.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/task2.root.20231206.051903.270403
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/task2.root.20231206.051903.270403/output
Streaming final output from /tmp/task2.root.20231206.051903.270403/output...
Removing temp directory /tmp/task2.root.20231206.051903.270403...


In [39]:
for file_name in file_names:
    input_file = 'hdfs://namenode:8020/' + file_name
    output_file = '/Hadoop_Result_'+ file_name
    !python3 task2.py -r hadoop $input_file --output $output_file

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/task2.root.20231206.052704.953780
uploading working dir files to hdfs:///user/root/tmp/mrjob/task2.root.20231206.052704.953780/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/task2.root.20231206.052704.953780/files/
Running step 1 of 2...
  Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
  packageJobJar: [/tmp/hadoop-unjar7448963435017902807/] [] /tmp/streamjob1205225726761750731.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.24.0.5:8032
  Connecting to ResourceManager at resourcemanager/172.24.0.5:8032
  Disabling

In [41]:
for file_name in file_names:
    input_file = 'hdfs://namenode:8020/Hadoop_Result_' + file_name + '/part-00000'
    output_file = 'Hadoop_Result_'+ file_name
    !hadoop fs -get $input_file $output_file

2023-12-06 08:30:14 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-12-06 08:30:15 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-12-06 08:30:17 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [44]:
!python3 task2.py -r hadoop hdfs://namenode:8020/SW_EpisodeIV.txt hdfs://namenode:8020/SW_EpisodeV.txt hdfs://namenode:8020/SW_EpisodeVI.txt --output /Hadoop_Result_EpisodeAll.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/task2.root.20231206.053236.940209
uploading working dir files to hdfs:///user/root/tmp/mrjob/task2.root.20231206.053236.940209/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/task2.root.20231206.053236.940209/files/
Running step 1 of 2...
  Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
  packageJobJar: [/tmp/hadoop-unjar7491208522132495394/] [] /tmp/streamjob3901650638494881949.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.24.0.5:8032
  Connecting to ResourceManager at resourcemanager/172.24.0.5:8032
  Disabling

In [45]:
!hadoop fs -get hdfs://namenode:8020/Hadoop_Result_EpisodeAll.txt/part-00000 Hadoop_Result_EpisodeAll.txt

2023-12-06 08:33:52 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [43]:
!hadoop fs -rm -r /Hadoop_Result_EpisodeAll.txt
# !hadoop fs -rm -r /Hadoop_Result_SW_EpisodeV.txt

2023-12-06 08:32:31 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted /Hadoop_Result_EpisodeAll.txt
